# 导入使用的包工具

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.metrics import pairwise_distances, pairwise_kernels
from sklearn.preprocessing import StandardScaler,LabelBinarizer
from sklearn.cluster import KMeans, SpectralClustering
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.sparse.linalg import eigs

# 导入鸢尾花数据集

In [2]:
iris = load_iris()
data = iris.data
labels = iris.target
scaler = StandardScaler()
data = scaler.fit_transform(data)
print(np.sum(labels==0),np.sum(labels==1),np.sum(labels==2))
np.random.seed(162)


50 50 50


# HB算法实现

In [3]:
import numpy as np

def generate_matrix(N):
    # 创建一个3*N的全零矩阵
    matrix = np.zeros((3, N))

    # 随机选择每列的一个元素，将其设置为1
    for i in range(N):
        matrix[np.random.randint(3), i] = 1

    # 检查每行的和，如果小于1，随机选择该行的一个元素，将其设置为1
    for i in range(3):
        if np.sum(matrix[i]) < 1:
            matrix[i, np.random.randint(N)] = 1
    return matrix


In [4]:
import numpy as np
import numpy as np
# 参数初始化
N = data.shape[0]
a = np.ones(N)
a = a*1/150
# 初始化U矩阵
u_init = generate_matrix(N)
u_init = generate_matrix(N)    
# print(np.sum(u_init,axis=0))

# 初始化收敛阈值

# 最大次数
T = 200

def euclidean_distance(a, b):
    return np.linalg.norm(a - b)

D = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        D[i, j] = euclidean_distance(data[i], data[j])

def calculate_alpha(U, a):
    return np.dot(U, a)

def calculate_d_i_k(a, U, D, i,k):
    alpha_i = calculate_alpha(U[i], a)
    return np.dot(a, U[i] * D[k]) / alpha_i

def calculate_d_i(a, U, D, i):
    N = a.shape[0]
    d_i = 0
    for k in range(N):
        d_i = a[k]*U[i][k]*calculate_d_i_k(a, U, D, i, k)

    return d_i 

def calculate_d(a, U, D):
    C = U.shape[0]
    d = 0
    
    for i in range(C):
        alpha = calculate_alpha(U[i], a)
        D_i =  calculate_d_i(a, U, D, i)
        d += alpha * D_i
    return d

# print(calculate_d(a, u_init, D))
def u_i_k(a, U, D, i, k, t):
    C = U.shape[0]
    d_i_k = calculate_d_i_k(a, U, D, i,k)
    d_i  = calculate_d_i(a, U, D, i)
    alpha_i = calculate_alpha(U[i], a)

    temp = np.sum([calculate_alpha(U[j], a) * np.exp(-1 * (2 * calculate_d_i_k(a, U, D, j,k) - calculate_d_i(a, U, D, j)) / t) for j in range(C)])
    
    return alpha_i * np.exp(-1 * (2 * d_i_k - d_i) / t) / temp


def get_new_U(a, U, D, t):
    C, N = U.shape
    new = np.zeros((C, N))
    for i in range(C):
        for k in range(N):
            new[i, k] = u_i_k(a, U, D, i, k, t)
    return new

def diffArr(A, B):
    return np.linalg.norm(A - B, ord='fro')

def Main(a, U, D, t, epsilon):
    c = 0
    while True:
        new = get_new_U(a, U, D, t)
        if diffArr(U, new) > epsilon and c <= T:
            c += 1
            U = new
            # print(U)
        else:
            print(c)
            return new
epsilon = 1e-1
U  = Main(a, u_init, D, 0.05, epsilon)
def find_max_index(U):
    # 找到每列最大值的索引
    max_index_list = np.argmax(U, axis=0)
    return max_index_list
print("Accuracy:", np.mean(labels == find_max_index(U)))



14
Accuracy: 0.8133333333333334


# SATM算法实现

In [5]:
iris = load_iris()
data = iris.data
labels = iris.target
scaler = StandardScaler()
data = scaler.fit_transform(data)
print(np.sum(labels==0),np.sum(labels==1),np.sum(labels==2))


50 50 50


In [7]:
import numpy as np
import numpy as np
# 参数初始化
N = data.shape[0]
a = np.ones(N)
a = a*1/150
# 初始化U矩阵


def euclidean_distance(a, b):
    return np.linalg.norm(a - b)

# 初始化相似性矩阵
N = data.shape[0]
S = np.zeros((N, N))

# 计算相似性矩阵
for i in range(N):
    for j in range(N):
        S[i, j] = euclidean_distance(data[i], data[j])
        if S[i,j] != 0:
            S[i,j]=1.0/S[i,j]


            
def calculate_alpha(U, a):
    return np.dot(U, a)

def calculate_s_i_k(a, U, S, i, k):
    alpha_i = calculate_alpha(U[i], a)
    return np.dot(a, U[i] * S[k]) / alpha_i

def calculate_s_i(a, U, S, i):
    N = a.shape[0]
    s_i = 0
    for k in range(N):
        s_i = a[k] * U[i][k] * calculate_s_i_k(a, U, S, i, k)
    return s_i

def calculate_s(a, U, S):
    C = U.shape[0]
    s = 0
    for i in range(C):
        alpha = calculate_alpha(U[i], a)
        S_i = calculate_s_i(a, U, S, i)
        s += alpha * S_i
    return s

def u_i_k(a, U, S, i, k, t):
    C = U.shape[0]
    s_i_k = calculate_s_i_k(a, U, S, i, k)
    s_i = calculate_s_i(a, U, S, i)
    alpha_i = calculate_alpha(U[i], a)

    temp = np.sum([
        calculate_alpha(U[j], a) * np.exp( (2 * calculate_s_i_k(a, U, S, j, k) - calculate_s_i(a, U, S, j)) / t)
        for j in range(C)
    ])

    return alpha_i * np.exp( (2 * s_i_k - s_i) / t) / temp

def filter(arr):
    min_values = np.nanmin(arr, axis=1, keepdims=True)
    nan_indices = np.isnan(arr)
    arr[nan_indices] = np.where(nan_indices, np.tile(min_values, (1, arr.shape[1])), arr)[nan_indices]
    return arr

def get_new_U(a, U, S, t):
    C, N = U.shape
    new = np.zeros((C, N))
    for i in range(C):
        for k in range(N):
            new[i, k] = u_i_k(a, U, S, i, k, t)
    return new

def diffArr(A, B):
    return np.linalg.norm(A - B)

def Main(a, U, S, t, epsilon):
    c = 0
    while True:
        new = get_new_U(a, U, S, t)
        if diffArr(U, new) > epsilon and c <=  T:
            c += 1
            print(diffArr(U, new))
            U = new
         
            # print(U)
        else:
            print(c)
            return new
epsilon = 1e-3

# 最大次数
T = 200
U  = Main(a, u_init, S, 0.1, epsilon)
print("Accuracy:", np.mean(labels == find_max_index(U)))


11.932320199994273
6.409522168678523
5.869585100194064
3.9404175759684237
1.6270563449769644
1.748502093225663
1.9053779146206233
1.52473688424037
2.068533779821144
3.2222438160089286
1.7287137800346504
1.8951172496255824
1.6790268758739446
1.0119546403590938
0.8884792965537014
0.7355706087395267
0.5937086015907613
0.43624961585552813
0.41717944452964556
0.5771851429174772
0.6044432816432617
0.3149290957847197
0.10362390833149561
0.03172847866792359
0.012308819415376742
0.005556757976124375
0.0025968911010500043
0.0012325673489720374
28
Accuracy: 0.8266666666666667


: 

# 自己的算法

In [ ]:
import numpy as np
import numpy as np
# 参数初始化
N = data.shape[0]
a = np.ones(N)
a = a*1/150
# 初始化U矩阵
# 最大次数
T = 200

def euclidean_distance(a, b):
    return np.linalg.norm(a - b)

# 初始化相似性矩阵
N = data.shape[0]
S = np.zeros((N, N))
D = np.zeros((N, N))
# 计算相似性矩阵
for i in range(N):
    for j in range(N):
        S[i, j] = euclidean_distance(data[i], data[j])
        if S[i,j]==0:
            D[i,j]=0
        else:
            D[i, j] = 1./S[i, j]

def calculate_alpha(U, a):
    return np.dot(U, a)

def calculate_s_i_k(a, U, S, i, k):
    alpha_i = calculate_alpha(U[i], a)
    return np.dot(a, U[i] * S[k]) / alpha_i

def calculate_s_i(a, U, S, i):
    N = a.shape[0]
    s_i = 0
    for k in range(N):
        s_i = a[k] * U[i][k] * calculate_s_i_k(a, U, S, i, k)
    return s_i

def calculate_s(a, U, S):
    C = U.shape[0]
    s = 0
    for i in range(C):
        alpha = calculate_alpha(U[i], a)
        S_i = calculate_s_i(a, U, S, i)
        s += alpha * S_i
    return s

def u_i_k(a, U, S, D,i, k, t):
    C = U.shape[0]
    s_i_k = calculate_s_i_k(a, U, S, i, k)
    s_i = calculate_s_i(a, U, S, i)
    alpha_i = calculate_alpha(U[i], a)
    d_i_k = calculate_d_i_k(a, U, D, i,k)
    d_i  = calculate_d_i(a, U, D, i)
    temp = np.sum([
        calculate_alpha(U[j], a) * np.exp(-1 * (2 * (calculate_s_i_k(a, U, S, j, k) - calculate_d_i_k(a, U, D, j, k))+(calculate_s_i(a, U, S, j)-calculate_d_i(a, U, D, j))) / t)
        for j in range(C)
    ])

    return alpha_i * np.exp(-1 * (2 * s_i_k - s_i) / t) / temp

def filter(arr):
    min_values = np.nanmin(arr, axis=1, keepdims=True)
    nan_indices = np.isnan(arr)
    arr[nan_indices] = np.where(nan_indices, np.tile(min_values, (1, arr.shape[1])), arr)[nan_indices]
    return arr

def get_new_U(a, U, S,D, t):
    C, N = U.shape
    new = np.zeros((C, N))
    for i in range(C):
        for k in range(N):
            new[i, k] = u_i_k(a, U, S,D, i, k, t)
    return new
def calculate_d_i_k(a, U, D, i,k):
    alpha_i = calculate_alpha(U[i], a)
    return np.dot(a, U[i] * D[k]) / alpha_i

def calculate_d_i(a, U, D, i):
    N = a.shape[0]
    d_i = 0
    for k in range(N):
        d_i = a[k]*U[i][k]*calculate_d_i_k(a, U, D, i, k)

    return d_i 
def diffArr(A, B):
    return np.linalg.norm(A - B, ord='fro')

def Main(a, U, S,D, t, epsilon):
    c = 0
    while True:
        new = get_new_U(a, U, S,D, t)
        if diffArr(U, new) > epsilon and c <= T:
            c += 1
            U = new
            # print(U)
        else:
            print(c)
            return U
epsilon = 1e-6
u_init = generate_matrix(N)   
U  = Main(a, u_init, S,D, 0.05, epsilon)
print("Accuracy:", np.mean(labels == find_max_index(U)))


7
Accuracy: 0.42


# 实验结果分析
在鸢尾花数据集中HB算法明显优于SATM算法
1. **算法原理**：
    - LATM：LATM算法是一种基于局部拓扑结构保持的聚类算法，它倾向于在局部区域内保持数据的拓扑结构，对数据集中的局部结构进行聚类。
    - HB：HB算法是一种基于层次划分的聚类方法，它通过递归地将数据集划分成更小的子集，然后在子集上进行聚类。
2. **数据集特点**：
    - LATM适用于保持局部拓扑结构，如果数据在局部区域内有明显的聚类结构或者类别边界清晰，LATM可能更有效。
    - HB算法在处理全局结构复杂且具有层次性的数据时可能表现更好。

